# NLU

In [1]:
import spacy
import json
import nltk
import nlpnet
from nltk.tokenize import sent_tokenize

/home/douglas/.virtualenvs/PLN-IA_env/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:17: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, defaultdict


In [2]:
nlp = spacy.load('pt_core_news_sm')
sent_tokenizer = nltk.data.load('tokenizers/punkt/portuguese.pickle')

srl_tagger = nlpnet.SRLTagger('srl-pt', language='pt')

In [24]:
def nlu(text):
    annotation = {"sentences": []}
    sentences = sent_tokenizer.tokenize(text)
    for sentence in sentences:
        ann = {"sentence": sentence, "tokens": [], "POS": [], "DEP": [], "NER": [], "SRL": [], "WA": []}
        doc = nlp(sentence)
        
        
        for token in doc:
            ann["tokens"].append(str(token.text))
            ann["POS"].append(str(token.pos_))
            ann["DEP"].append((str(token.dep_), str(token.head)))
            ann["NER"].append(str(token.ent_type_))
                
            if(str(token.pos_)=="NOUN" or str(token.pos_)=="ADJ" or str(token.pos_)=="PROPN"):
                ann["WA"].append(str(token.text))
        
        i = 0
        while (i < len(ann["POS"])):
            propn_token = ""
            if((ann["POS"][i]=="NOUN" and ann["POS"][i+1]=="ADJ") or
               (ann["POS"][i]=="ADJ" and ann["POS"][i+1]=="NOUN")):
                ann["WA"].append(ann["tokens"][i]+" "+ann["tokens"][i+1])
            
            if(ann["POS"][i]=="PROPN" and ann["POS"][i+1]=="PROPN"):
                propn_token += ann["tokens"][i] + " "
                while(ann["POS"][i+1]=="PROPN"):                                                               
                    i+=1
                    propn_token+= ann["tokens"][i] + " "
                                                   
                ann["WA"].append(propn_token.strip())
                
            if(ann["POS"][i]=="NOUN" and ann["POS"][i+1]=="NOUN"):
                propn_token += ann["tokens"][i] + " "
                while(ann["POS"][i+1]=="NOUN"):                                                               
                    i+=1
                    propn_token+= ann["tokens"][i] + " "
                                                   
                ann["WA"].append(propn_token.strip())
                                    
            i+=1
            
        print(ann["WA"])
        sent = srl_tagger.tag(u''+sentence)[0]
        srl = []
        
        for s in sent.arg_structures:
            
            for key, value in s[1].items():
                temp={key: value}
                srl.append(temp)
            
        ann["SRL"].append(srl)
        annotation["sentences"].append(ann)
    return json.dumps(annotation)

In [25]:
text = """Dando continuidade à programação do XXXII Congresso de Iniciação Científica (Ciufla), do XIV Congresso de Extensão (Conex) e do XXVIII Congresso da Pós-Graduação, foram realizadas palestras com professores de diferentes áreas da UFLA com o intuito de mostrar como é possível captar recursos de empresas privadas por meio de parcerias e ainda levar tecnologias geradas na Universidade para fora do ambiente acadêmico João Silva.

O professor Júlio Louzada Silva Santos, do Departamento de Biologia (DBI), mostrou como é importante buscar a qualidade das publicações das pesquisas desenvolvidas na UFLA, e os professores Flávio Borém, do Departamento de Engenharia (DEG), e Carla Ávila, do Departamento de Zootecnia (DZO), falaram sobre como as ideias inovadoras ajudam a atrair empresas parceiras para investimentos na Universidade."""
nlu(text)

['continuidade', 'programação', 'XXXII', 'Congresso', 'Iniciação', 'Científica', 'Ciufla', 'XIV', 'Congresso', 'Extensão', 'Conex', 'XXVIII', 'Congresso', 'Pós-Graduação', 'palestras', 'professores', 'áreas', 'UFLA', 'intuito', 'possível', 'recursos', 'empresas', 'meio', 'parcerias', 'tecnologias', 'Universidade', 'ambiente', 'acadêmico', 'João', 'Silva', 'XXXII Congresso', 'Iniciação Científica', 'ambiente acadêmico', 'João Silva']
['professor', 'Júlio', 'Louzada', 'Silva', 'Santos', 'Departamento', 'Biologia', 'DBI', 'importante', 'qualidade', 'publicações', 'pesquisas', 'UFLA', 'Flávio', 'Borém', 'Departamento', 'Engenharia', 'DEG', 'Carla', 'Ávila', 'Departamento', 'Zootecnia', 'DZO', 'como', 'ideias', 'inovadoras', 'empresas', 'parceiras', 'investimentos', 'Universidade', 'Júlio Louzada Silva Santos', 'Flávio Borém', 'Carla Ávila', 'ideias inovadoras', 'empresas parceiras']


'{"sentences": [{"sentence": "Dando continuidade \\u00e0 programa\\u00e7\\u00e3o do XXXII Congresso de Inicia\\u00e7\\u00e3o Cient\\u00edfica (Ciufla), do XIV Congresso de Extens\\u00e3o (Conex) e do XXVIII Congresso da P\\u00f3s-Gradua\\u00e7\\u00e3o, foram realizadas palestras com professores de diferentes \\u00e1reas da UFLA com o intuito de mostrar como \\u00e9 poss\\u00edvel captar recursos de empresas privadas por meio de parcerias e ainda levar tecnologias geradas na Universidade para fora do ambiente acad\\u00eamico Jo\\u00e3o Silva.", "tokens": ["Dando", "continuidade", "\\u00e0", "programa\\u00e7\\u00e3o", "do", "XXXII", "Congresso", "de", "Inicia\\u00e7\\u00e3o", "Cient\\u00edfica", "(", "Ciufla", ")", ",", "do", "XIV", "Congresso", "de", "Extens\\u00e3o", "(", "Conex", ")", "e", "do", "XXVIII", "Congresso", "da", "P\\u00f3s-Gradua\\u00e7\\u00e3o", ",", "foram", "realizadas", "palestras", "com", "professores", "de", "diferentes", "\\u00e1reas", "da", "UFLA", "com", "o", "int